In [1]:
import pandas as pd
import datetime
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
import smtplib
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email import encoders
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
import time
import pandas as pd
import pandas as pd
from datetime import datetime, timedelta
from tkinter import *
from tkinter import ttk


In [125]:
###FUNCTION
def web_content_div(web_content, class_path):
    web_content_div = web_content.find_all('div',{'class': 'D(ib) Mend(20px)'})
    try:
        spans = web_content_div[0].find('span')
        texts = [spans.get_text() for span in spans]
    except IndexError:
        texts = []
    return texts


def real_time_price(ticker):
    global change
    url = "https://finance.yahoo.com/quote/UCG.MI/"
    r = requests.get(url, headers={'USER-AGENT': "Mozilla/5.0"})
    time.sleep(2)
    web_content = BeautifulSoup(r.text, 'lxml')
    time.sleep(2)
    try:
        curr_price = float(str(web_content.find_all('div',{'class': 'D(ib) Mend(20px)'})).split('value="')[1].split('"')[0])
    except:
        curr_price = None
    texts = web_content_div(web_content, 'D(ib) Mend(20px)')
    try:
        change = float(texts[0])
        if texts == []:
            texts = 0   
    except:
        change = None
    try:
        previous_price = curr_price - change
        perc_change = round(((curr_price/previous_price)-1)*100,2)
    except:
        previous_price = None
        perc_change = None
    return curr_price, change, previous_price, perc_change
        

class EmailSending():
    def launch_email_drop(self, perc_change):
        now = str(datetime.now())[0:19]
        gmail_user = 'zolimatteo2@gmail.com'
        gmail_password = 'zthk cocy ukjp zvof'

        sent_from = 'Matteo Zoli'
        to = e_mail_list
        subject = "Price drop by {}% at {} for stock: {}".format(perc_change, now, dictionary_tickers[stock])
        body =  "The stock {} is dropping at {}.\n Check it what is going on website:https://finance.yahoo.com/quote/{}/ ".format(dictionary_tickers[stock], now, stock)

        email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            server.sendmail(sent_from, to, email_text)
            server.close()

            print('Email sent!')
        except:
            print('Something went wrong...')


    def launch_email_up(self):
        now = str(datetime.now())[0:19]
        gmail_user = 'zolimatteo2@gmail.com'
        gmail_password = 'zthk cocy ukjp zvof'

        sent_from = 'Matteo Zoli'
        to = e_mail_list
        subject = "Price up by {}% at {} for stock: {}".format(price_drop, now, dictionary_tickers[stock])
        body =  "Good news!\nThe stock {} is going up by at {}.\n\n Check it what is going on website:https://finance.yahoo.com/quote/{}/ ".format(dictionary_tickers[stock], now, stock)

        email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            server.sendmail(sent_from, to, email_text)
            server.close()

            print('Email sent!')
        except:
            print('Something went wrong...')

ef = EmailSending()

In [126]:
#IMPORTING THE TICKERS
df_tickers_dataset = pd.read_excel("INPUT/yticker.xlsx")
df_tickers = df_tickers_dataset.rename(columns = {"Unnamed: 1": "Name", "Unnamed: 3": "Industry", "Unnamed: 4":"Country"})
##filtering from the first letter A and filter fr null values in name
df_tickers = df_tickers.iloc[3:][:].sort_values(by = 'Name')[1291:]
df_tickers["concatenated name ticker"] = df_tickers["Name"] + ' TICK: ' + df_tickers["Yahoo Stock Tickers"]
df_tickers = df_tickers[~df_tickers["Name"].isna()]
df_tickers = df_tickers.reset_index(drop = True)

dictionary_tickers = df_tickers.set_index('Yahoo Stock Tickers')['Name'].to_dict()



In [127]:
l = df_tickers["concatenated name ticker"][:].to_list()

def checkkey(event):
    value = event.widget.get()    
    print(value)
    # get data from l
    if value == '':
        data = l
    else:
        data = []
        for item in l:
            if value.lower() in item.lower():
                data.append(item)                
    update(data)
    
def update(data):
    lb.delete(0, 'end')
    for item in data:
        lb.insert('end', item)

# GUI
root = Tk()
root.title("Input Datas for the Financial Analysis")
bg = PhotoImage(file = "INPUT/background.png", master = root)
label1 = Label( root, image = bg)
label1.place(x = 0, y = 0)
root.geometry('600x700')
def login():
    print(input_entry.get())
    return var

def myClick():
    submission = "You succesfully submitted!\n Please now exit the window with X command"
    myLabel = Label(root, text=submission)
    myLabel.pack()

label_input = Label(root, text= "Welcome to your customised Financial Analyzer!"
                   "\nPlease read carefully the istructions to upload the datas the tool will need!"
                     "\nAnd, overall don't forget to have fun! Matteo ;)")
label_input.pack(padx=10, pady=10)
label_input = Label(root, text= "Select the frequency for which the check will be done (in minutes)")
label_input.pack(padx=5, pady=5)
lista = [0.5, 1,5,15,30,60,120]
minutes_selected = StringVar()
minutes_combobox = ttk.Combobox(root, textvariable = minutes_selected)
minutes_combobox["values"] = [m for m in lista]
minutes_combobox["state"] = "readonly"
minutes_combobox.pack(padx=5, pady=5)
###DROP
label_input = Label(root, text= "Signal when drop -% occurs")
label_input.pack(padx=5, pady=5)
drop = StringVar()
input_entry = ttk.Entry(root, textvariable = drop)
input_entry.pack(padx=5, pady=5)
###GROWTH
label_input = Label(root, text= "Signal when growth +% occurs")
label_input.pack(padx=5, pady=5)
growth = StringVar()
input_entry = ttk.Entry(root, textvariable = growth)
input_entry.pack(padx=5, pady=5)

###STOP AFTER N MINUTES
label_input = Label(root, text= "Stop the analysis after minutes :")
label_input.pack(padx=5, pady=5)
lista = [5,15,30,60,120,180,240,300, 400]
stop_minutes = StringVar()
stop_minutes_combobox = ttk.Combobox(root, textvariable = stop_minutes)
stop_minutes_combobox["values"] = [m for m in lista]
stop_minutes_combobox["state"] = "readonly"
stop_minutes_combobox.pack(padx=5, pady=5)
#input_entry.pack(padx=10, pady=10)


label_input = Label(root, text= "Search, find and type or click the stocks you are interested in."
                    "\nClick the button every time you want to add the stock")
label_input.pack(padx=10, pady=10)
tickers_selected_string = StringVar()
lb = Listbox(root, width=40, height=10, selectmode=MULTIPLE)
list_selected = []
# Function for printing the selected listbox value(s)
def selected_item():
    # Traverse the tuple returned by  curselection method and print corresponding value(s) in the listbox
    for i in lb.curselection():
        print(lb.get(i))
        list_selected.append(lb.get(i))
    return list_selected

#creating text box 
e = Entry(root)
e.pack()
e.bind('<KeyRelease>', checkkey)
#creating list box
update(l)
lb.pack()

label_input = Label(root, text= "Insert list of e-mail separated by comma (i.e.: ,)")
label_input.pack(padx=10, pady=10)
e_mail = StringVar()
input_entry = ttk.Entry(root, textvariable = e_mail)
input_entry.pack()


btn = Button(root, text='Add to the list selected', command=selected_item)
# Placing the button and listbox
btn.pack(side='bottom')

root.mainloop()
if drop.get() != '':
    drop_value = float(drop.get())
    if drop_value > 0:
        drop_value = -drop_value
    else:
        drop_value = drop_value
else:
    drop_value = -5
if growth.get() != '':
    growth_value = float(growth.get())

    if growth_value < 0:
        growth_value = -growth_value
    else:
        growth_value = growth_value
else:
    growth_value = +5

if stop_minutes.get() != '':
    stop_minutes_values = int(stop_minutes.get())
else:
    stop_minutes = 120
if minutes_selected.get() == "":
    minutes = 1
else:
    minutes = int(minutes_selected.get())
e_mail_values = e_mail.get()
e_mail_list = e_mail_values.split(",")
##make unique items
list_selected_to_be_cleaned_from_ticker = list(set(list_selected))
##take the ticker
list_selected = []
for i in list_selected_to_be_cleaned_from_ticker:
    i = i.split(" TICK: ")[1]
    list_selected.append(i)

u
un
uni
unic
unicr
unicre
unicred
unicredi
UniCredit S.p.A. TICK: UCG.MI
UniCredit S.p.A. TICK: UCG.MI


In [128]:
#PROCESSING AND SENDING
t_end = time.time() + stop_minutes_values*60
while time.time() < t_end:
    print("stop_minutes_values:", stop_minutes_values)
    print("minutes:", minutes)
    no_warning = []
    no_datas = []
    for stock in list_selected:
        print(stock)
        curr_price, change, previous_price, perc_change  = real_time_price(stock)
        print(curr_price, change, previous_price, perc_change)
        price_up= growth_value
        #price_drop must be negative
        price_drop= drop_value
        if curr_price!= None and perc_change!= None and price_drop!= None:
            if perc_change < price_drop:
                print("launching e-mail for stock {} drops".format(stock))
                ef.launch_email_drop(perc_change)    
            elif perc_change > price_up:
                print("launching e-mail for stock {} is up".format(stock))
                ef.launch_email_up()    
            else:
                no_warning.append(stock) 
        else:
            no_datas.append(stock)
    print("List of tickers:", list_selected)        
    print("List of tickers that did not change much:", no_warning)
    print("List of tickers that did could not be assessed because of lack of datas:", no_datas)

    time.sleep(minutes*60)
print("we are done")

stop_minutes_values: 5
minutes: 1
UCG.MI
23.345 -0.32 23.665 -1.35
launching e-mail for stock UCG.MI drops
Email sent!
List of tickers: ['UCG.MI']
List of tickers that did not change much: []
List of tickers that did could not be assessed because of lack of datas: []
stop_minutes_values: 5
minutes: 1
UCG.MI
23.345 -0.32 23.665 -1.35
launching e-mail for stock UCG.MI drops
Email sent!
List of tickers: ['UCG.MI']
List of tickers that did not change much: []
List of tickers that did could not be assessed because of lack of datas: []
stop_minutes_values: 5
minutes: 1
UCG.MI
23.345 -0.32 23.665 -1.35
launching e-mail for stock UCG.MI drops
Email sent!
List of tickers: ['UCG.MI']
List of tickers that did not change much: []
List of tickers that did could not be assessed because of lack of datas: []


KeyboardInterrupt: 